In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [ ]:
class DataFrameConverter:
    
    @staticmethod
    def convertColumnsOfDataFrameToNumerics(dataFrame, columns):
        for column in columns:
            DataFrameConverter._convertColumnOfDataFrameToNumeric(dataFrame, column)

    @staticmethod
    def _convertColumnOfDataFrameToNumeric(dataFrame, column):
        dataFrame[column] = np.where(dataFrame[column] == 'Y', 1, 0)
    

In [ ]:
import pandas as pd

class VaersDescrReader:
    
    def __init__(self, dataDir):
        self.dataDir = dataDir        

    def readAllVaersDescrs(self):
        return self.readVaersDescrs(["2021", "2022"])
        
    def readVaersDescrs(self, years):
        return [self.readVaersDescr(year) for year in years]

    def readVaersDescr(self, year):
        folder = self.dataDir + "/" + year + "VAERSData/"
        return {
                    'VAERSDATA': self._readVAERSDATA(folder + year + "VAERSDATA.csv"),
                    'VAERSVAX': self._readVAERSVAX(folder + year + "VAERSVAX.csv")
               }

    def _readVAERSDATA(self, file):
        VAERSDATA = self._read_csv(
            file = file,
            usecols = ['VAERS_ID', 'RECVDATE', 'DIED', 'L_THREAT', 'DISABLE', 'HOSPITAL', 'ER_VISIT'],
            parse_dates = ['RECVDATE'],
            date_parser = lambda dateStr: pd.to_datetime(dateStr, format = "%m/%d/%Y"))
        DataFrameConverter.convertColumnsOfDataFrameToNumerics(
            VAERSDATA,
            ['DIED', 'L_THREAT', 'DISABLE', 'HOSPITAL', 'ER_VISIT'])
        return VAERSDATA

    def _readVAERSVAX(self, file):
        return self._read_csv(
            file = file,
            usecols = ['VAERS_ID', 'VAX_DOSE_SERIES', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT'],
            dtype = {"VAX_DOSE_SERIES": "string"})

    def _read_csv(self, file, **kwargs):
        return pd.read_csv(
            file,
            index_col = 'VAERS_ID',
            encoding = 'latin1',
            low_memory = False,
            **kwargs)


In [ ]:
import pandas as pd

class VaersDescr2DataFrameConverter:

    @staticmethod
    def createDataFrameFromDescr(vaersDescr):
        return pd.merge(
                vaersDescr['VAERSDATA'],
                vaersDescr['VAERSVAX'],
                how = 'left',
                left_index = True,
                right_index = True,
                validate = 'one_to_many')

    @staticmethod
    def createDataFrameFromDescrs(vaersDescrs):
        dataFrames = [VaersDescr2DataFrameConverter.createDataFrameFromDescr(vaersDescr) for vaersDescr in vaersDescrs]
        return pd.concat(dataFrames)


In [ ]:
import pandas as pd

class DataFrameFilter:
    
    def filterByCovid19(self, dataFrame):
        return dataFrame[self._isCovid19(dataFrame)]

    def filterBy(self, dataFrame, manufacturer = None, dose = None):
        return dataFrame[self._isManufacturer(dataFrame, manufacturer) & self._isDose(dataFrame, dose)]

    def _isCovid19(self, dataFrame):
        return dataFrame["VAX_TYPE"] == "COVID19"

    def _isManufacturer(self, dataFrame, manufacturer):
        return dataFrame["VAX_MANU"] == manufacturer if manufacturer is not None else True

    def _isDose(self, dataFrame, dose):
        return dataFrame["VAX_DOSE_SERIES"].str.contains(dose) if dose is not None else True


In [ ]:
class AggregationHelper:

    @staticmethod
    def aggregateAndFlattenColumnsAndRenameColumns(dataFrame, aggFunctionsByColumn, columnNameMappingsDict):
        aggregatedDataFrame = dataFrame.agg(aggFunctionsByColumn)
        AggregationHelper._flattenColumns(aggregatedDataFrame)
        return aggregatedDataFrame.rename(columns = columnNameMappingsDict)

    @staticmethod
    def _flattenColumns(dataFrame):
        dataFrame.columns = ["_".join(a) for a in dataFrame.columns.to_flat_index()]


In [ ]:
import pandas as pd

class BatchCodeTableHelper:
    
    def __init__(self, dataFrame : pd.DataFrame):
        self.dataFrame = dataFrame        

    def createBatchCodeTable(self):
        batchCodeTable = AggregationHelper.aggregateAndFlattenColumnsAndRenameColumns(
            dataFrame = self.dataFrame.groupby('VAX_LOT'),
            aggFunctionsByColumn = {
                'DIED': ['sum', 'size'],
                'L_THREAT': 'sum',
                'DISABLE': 'sum'
            },
            columnNameMappingsDict = {
                "DIED_size": "ADRs",
                "DIED_sum": "DEATHS",
                "L_THREAT_sum": "LIFE THREATENING ILLNESSES",
                "DISABLE_sum": "DISABILITIES"
            })
        batchCodeTable = batchCodeTable[['ADRs', 'DEATHS', 'DISABILITIES', 'LIFE THREATENING ILLNESSES']]
        return batchCodeTable.sort_values(by = 'ADRs', ascending = False)

    # create table from https://www.howbadismybatch.com/combined.html
    def createSevereEffectsBatchCodeTable(self):
        batchCodeTable = AggregationHelper.aggregateAndFlattenColumnsAndRenameColumns(
            dataFrame = self.dataFrame.groupby('VAX_LOT'),
            aggFunctionsByColumn = {
                'DIED': ['sum', 'size'],
                'L_THREAT': 'sum',
                'DISABLE': 'sum',
                'HOSPITAL': 'sum',
                'ER_VISIT': 'sum'
            },
            columnNameMappingsDict = {
                "DIED_size": "ADRs",
                "DIED_sum": "DEATHS",
                "L_THREAT_sum": "LIFE THREATENING ILLNESSES",
                "DISABLE_sum": "DISABILITIES",
                'HOSPITAL_sum': 'HOSPITALISATIONS',
                'ER_VISIT_sum': 'EMERGENCY ROOM OR DOCTOR VISITS'
            })
        batchCodeTable = batchCodeTable[['ADRs', 'DEATHS', 'DISABILITIES', 'LIFE THREATENING ILLNESSES', 'HOSPITALISATIONS', 'EMERGENCY ROOM OR DOCTOR VISITS']]
        batchCodeTable = batchCodeTable.sort_values(by = 'ADRs', ascending = False)
        return self._addCompanyColumn(batchCodeTable, self._createCompanyByBatchCodeTable())

    def _addCompanyColumn(self, batchCodeTable, companyByBatchCodeTable):
        return pd.merge(
            batchCodeTable,
            companyByBatchCodeTable,
            how = 'left',
            left_index = True,
            right_index = True,
            validate = 'one_to_one')

    def _createCompanyByBatchCodeTable(self):
        return self._createManufacturerByBatchCodeTable().rename(columns = {"VAX_MANU": "COMPANY"})

    def _createManufacturerByBatchCodeTable(self):
        manufacturerByBatchCodeTable = self.dataFrame[['VAX_LOT', 'VAX_MANU']]
        manufacturerByBatchCodeTable = manufacturerByBatchCodeTable.drop_duplicates(subset = ['VAX_LOT'])
        return manufacturerByBatchCodeTable.set_index('VAX_LOT')


class BatchCodeTableFactory:

    @staticmethod
    def createBatchCodeTable(dataFrame : pd.DataFrame, manufacturer, dose):
        dataFrame = DataFrameFilter().filterByCovid19(dataFrame)
        filteredDataFrame = DataFrameFilter().filterBy(dataFrame, manufacturer = manufacturer, dose = dose)
        return BatchCodeTableHelper(filteredDataFrame).createBatchCodeTable()

    # create table from https://www.howbadismybatch.com/combined.html
    @staticmethod
    def createSevereEffectsBatchCodeTable(dataFrame : pd.DataFrame, dose):
        dataFrame = DataFrameFilter().filterByCovid19(dataFrame)
        dataFrame = DataFrameFilter().filterBy(dataFrame, dose = dose)
        return BatchCodeTableHelper(dataFrame).createSevereEffectsBatchCodeTable()


In [ ]:
class DoseAnalysis:
    
    @staticmethod
    def getDoseTable(dataFrame):
        return DoseAnalysis._getDoseTable(
            dataFrame.groupby(
                dataFrame['VAX_DOSE_SERIES'].rename('Dose')))

    @staticmethod
    def getDoseByMonthTable(dataFrame):
        return DoseAnalysis._getDoseTable(
            dataFrame.groupby(
                [
                    dataFrame['RECVDATE'].dt.year.rename('Year'),
                    dataFrame['RECVDATE'].dt.month.rename('Month'),
                    dataFrame['VAX_DOSE_SERIES'].rename('Dose')
                ]))

    @staticmethod
    def _getDoseTable(dataFrame):
        doseTable = AggregationHelper.aggregateAndFlattenColumnsAndRenameColumns(
            dataFrame = dataFrame,
            aggFunctionsByColumn = {
                'DIED': ['sum', 'size'],
                'L_THREAT': 'sum',
                'DISABLE': 'sum'
            },
            columnNameMappingsDict = {
                "DIED_size": "Total reports",
                "DIED_sum": "Deaths",
                "L_THREAT_sum": "Life Threatening Illnesses",
                "DISABLE_sum": "Disabilities"
            })
        doseTable = doseTable[['Total reports', 'Deaths', 'Disabilities', 'Life Threatening Illnesses']]
        doseTable['Severe reports (%)'] = (doseTable['Deaths'] + doseTable['Disabilities'] + doseTable['Life Threatening Illnesses']) / doseTable['Total reports'] * 100
        return doseTable


In [ ]:
import unittest

In [ ]:
from pandas.testing import assert_frame_equal

class DataFrameFilterTest(unittest.TestCase):

    def test_filterByCovid19_filterBy(self):
        # Given
        dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
            [
                {
                    'VAERSDATA': self.createDataFrame(
                        columns = ['DIED', 'L_THREAT', 'DISABLE'],
                        data = [  [1,      0,          0],
                                  [0,      0,          1]],
                        index = [
                            "0916600",
                            "0916601"]),
                    'VAERSVAX': self.createDataFrame(
                        columns = ['VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
                        data = [  ['COVID19',  'MODERNA',  '037K20A', '1'],
                                  ['COVID19',  'MODERNA',  '025L20A', '1']],
                        index = [
                            "0916600",
                            "0916601"],
                        dtypes = {'VAX_DOSE_SERIES': "string"})
                },
                {
                        'VAERSDATA': self.createDataFrame(
                        columns = ['DIED', 'L_THREAT', 'DISABLE'],
                        data = [  [0,       0,         0],
                                  [0,       0,         1]],
                            index = [
                            "1996873",
                            "1996874"]),
                        'VAERSVAX': self.createDataFrame(
                            columns = ['VAX_TYPE', 'VAX_MANU',         'VAX_LOT', 'VAX_DOSE_SERIES'],
                            data = [  ['HPV9',     'MERCK & CO. INC.', 'R017624', 'UNK'],
                                      ['COVID19',  'MODERNA',          '025L20A', '1']],
                            index = [
                                "1996873",
                                "1996874"],
                            dtypes = {'VAX_DOSE_SERIES': "string"})
                    }
            ])
        dataFrameFilter = DataFrameFilter()
            
        # When
        dataFrame = dataFrameFilter.filterByCovid19(dataFrame)
        dataFrame = dataFrameFilter.filterBy(dataFrame, manufacturer = "MODERNA", dose = '1')
        
        # Then
        dataFrameExpected = self.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
            data = [  [1,       0,         0,         'COVID19',  'MODERNA',  '037K20A', '1'],
                      [0,       0,         1,         'COVID19',  'MODERNA',  '025L20A', '1'],
                      [0,       0,         1,         'COVID19',  'MODERNA',  '025L20A', '1']],
            index = [
                "0916600",
                "0916601",
                "1996874"],
            dtypes = {'VAX_DOSE_SERIES': "string"})
        assert_frame_equal(dataFrame, dataFrameExpected, check_dtype = False)

    def test_filterByDose(self):
        # Given
        dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
            [
                {
                    'VAERSDATA': self.createDataFrame(
                        columns = ['DIED', 'L_THREAT', 'DISABLE', 'HOSPITAL', 'ER_VISIT'],
                        data = [  [1,      1,          0,         1,          1],
                                  [0,      0,          1,         0,          1]],
                        index = [
                            "0916600",
                            "0916601"]),
                    'VAERSVAX': self.createDataFrame(
                        columns = ['VAX_TYPE', 'VAX_MANU',        'VAX_LOT', 'VAX_DOSE_SERIES'],
                        data = [  ['COVID19',  'MODERNA',         '037K20A', '1'],
                                  ['COVID19',  'PFIZER\BIONTECH', '025L20A', '1']],
                        index = [
                            "0916600",
                            "0916601"],
                        dtypes = {'VAX_DOSE_SERIES': "string"})
                }
            ])
        dataFrameFilter = DataFrameFilter()
        dataFrame = dataFrameFilter.filterByCovid19(dataFrame)

        # When
        dataFrame = dataFrameFilter.filterBy(dataFrame, dose = '1')
        
        # Then
        dataFrameExpected = self.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE', 'HOSPITAL', 'ER_VISIT', 'VAX_TYPE', 'VAX_MANU',        'VAX_LOT', 'VAX_DOSE_SERIES'],
            data = [  [1,      1,          0,         1,          1,          'COVID19',  'MODERNA',         '037K20A', '1'],
                      [0,      0,          1,         0,          1,          'COVID19',  'PFIZER\BIONTECH', '025L20A', '1']],
            index = [
                "0916600",
                "0916601"],
            dtypes = {'VAX_DOSE_SERIES': "string"})
        assert_frame_equal(dataFrame, dataFrameExpected, check_dtype = False)

    def test_filterByFirstDose(self):
        # Given
        dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
            [
                {
                    'VAERSDATA': self.createDataFrame(
                        columns = ['DIED', 'L_THREAT', 'DISABLE'],
                        data = [  [1,      0,          0]],
                        index = [
                            "1048786"]),
                    'VAERSVAX': self.createDataFrame(
                        columns = ['VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
                        data = [  ['COVID19',  'MODERNA',  '016M20A', '2'],
                                  ['COVID19',  'MODERNA',  '030L20A', '1']],
                        index = [
                            "1048786",
                            "1048786"],
                        dtypes = {'VAX_DOSE_SERIES': "string"})
                }
            ])
        dataFrameFilter = DataFrameFilter()
            
        # When
        dataFrame = dataFrameFilter.filterByCovid19(dataFrame)
        dataFrame = dataFrameFilter.filterBy(dataFrame, manufacturer = "MODERNA", dose = '1')
        
        # Then
        dataFrameExpected = self.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
            data = [  [1,      0,          0,         'COVID19',  'MODERNA',  '030L20A', '1']],
            index = [
                "1048786"],
            dtypes = {'VAX_DOSE_SERIES': "string"})
        assert_frame_equal(dataFrame, dataFrameExpected, check_dtype = False)

    def test_filterBySecondDose(self):
        # Given
        dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
            [
                {
                    'VAERSDATA': self.createDataFrame(
                        columns = ['DIED', 'L_THREAT', 'DISABLE'],
                        data = [  [1,      0,          0]],
                        index = [
                            "1048786"]),
                    'VAERSVAX': self.createDataFrame(
                        columns = ['VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
                        data = [  ['COVID19',  'MODERNA',  '016M20A',  '2'],
                                  ['COVID19',  'MODERNA',  '030L20A',  '1']],
                        index = [
                            "1048786",
                            "1048786"],
                        dtypes = {'VAX_DOSE_SERIES': "string"})
                }
            ])
        dataFrameFilter = DataFrameFilter()

        # When
        dataFrame = dataFrameFilter.filterByCovid19(dataFrame)
        dataFrame = dataFrameFilter.filterBy(dataFrame, manufacturer = "MODERNA", dose = '2')
        
        # Then
        dataFrameExpected = self.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
            data = [  [1,      0,          0,         'COVID19',  'MODERNA',  '016M20A', '2']],
            index = [
                "1048786"],
            dtypes = {'VAX_DOSE_SERIES': "string"})
        assert_frame_equal(dataFrame, dataFrameExpected, check_dtype = False)

    def createDataFrame(self, index, columns, data, dtypes = {}):
        return pd.DataFrame(index = index, columns = columns, data = data).astype(dtypes)


In [ ]:
from pandas.testing import assert_frame_equal

class BatchCodeTableFactoryTest(unittest.TestCase):

    def test_createSevereEffectsBatchCodeTable(self):
        # Given
        dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
            [
                {
                    'VAERSDATA': self.createDataFrame(
                        columns = ['DIED', 'L_THREAT', 'DISABLE', 'HOSPITAL', 'ER_VISIT'],
                        data = [  [1,      1,          0,         1,          1],
                                  [0,      0,          1,         0,          1]],
                        index = [
                            "0916600",
                            "0916601"]),
                    'VAERSVAX': self.createDataFrame(
                        columns = ['VAX_TYPE', 'VAX_MANU',        'VAX_LOT', 'VAX_DOSE_SERIES'],
                        data = [  ['COVID19',  'MODERNA',         '037K20A', '1'],
                                  ['COVID19',  'PFIZER\BIONTECH', '025L20A', '1']],
                        index = [
                            "0916600",
                            "0916601"],
                        dtypes = {'VAX_DOSE_SERIES': "string"})
                }
            ])

        # When
        batchCodeTable = BatchCodeTableFactory.createSevereEffectsBatchCodeTable(dataFrame, '1')

        # Then
        batchCodeTableExpected = pd.DataFrame(
            data = {
                'ADRs': [1, 1],
                'DEATHS': [0, 1],
                'DISABILITIES': [1, 0],
                'LIFE THREATENING ILLNESSES': [0, 1],
                'HOSPITALISATIONS': [0, 1],
                'EMERGENCY ROOM OR DOCTOR VISITS': [1, 1],
                'COMPANY': ['PFIZER\BIONTECH', 'MODERNA']
            },
            index = pd.Index(['025L20A', '037K20A'], name = 'VAX_LOT'))
        assert_frame_equal(batchCodeTable, batchCodeTableExpected, check_dtype = False)

    def test_createBatchCodeTable2(self):
        dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
            [
               {
                   'VAERSDATA': self.createDataFrame(
                       columns = ['DIED', 'L_THREAT', 'DISABLE', 'HOSPITAL', 'ER_VISIT'],
                       data = [  [1,      0,          0,         0,          0],
                                 [0,      0,          1,         0,          0]],
                       index = [
                           "0916600",
                           "0916601"]),
                   'VAERSVAX': self.createDataFrame(
                       columns = ['VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
                       data = [  ['COVID19',  'MODERNA',  '037K20A', '1'],
                                 ['COVID19',  'MODERNA',  '025L20A', '1']],
                       index = [
                           "0916600",
                           "0916601"],
                       dtypes = {'VAX_DOSE_SERIES': "string"})
               },
               {
                    'VAERSDATA': self.createDataFrame(
                       columns = ['DIED', 'L_THREAT', 'DISABLE', 'HOSPITAL', 'ER_VISIT'],
                       data = [  [0,      0,          0,         0,          0],
                                 [0,      0,          1,         0,          0]],
                       index = [
                           "1996873",
                           "1996874"]),
                    'VAERSVAX': self.createDataFrame(
                        columns = ['VAX_TYPE', 'VAX_MANU',         'VAX_LOT', 'VAX_DOSE_SERIES'],
                        data = [  ['HPV9',     'MERCK & CO. INC.', 'R017624', 'UNK'],
                                  ['COVID19',  'MODERNA',          '025L20A', '1']],
                        index = [
                            "1996873",
                            "1996874"],
                        dtypes = {'VAX_DOSE_SERIES': "string"})
                }
            ])
        self._test_createBatchCodeTable(dataFrame, "MODERNA", '1')

    def test_createBatchCodeTable(self):
        dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
            VaersDescrReader(dataDir = "test/VAERS").readAllVaersDescrs())
        self._test_createBatchCodeTable(dataFrame, "MODERNA", '1')

    def _test_createBatchCodeTable(self, dataFrame, manufacturer, dose):
        # When
        batchCodeTable = BatchCodeTableFactory.createBatchCodeTable(dataFrame, manufacturer, dose)

        # Then
        batchCodeTableExpected = pd.DataFrame(
            data = {
                'ADRs': [2, 1],
                'DEATHS': [0, 1],
                'DISABILITIES': [2, 0],
                'LIFE THREATENING ILLNESSES': [0, 0]
            },
            index = pd.Index(['025L20A', '037K20A'], name = 'VAX_LOT'))
        assert_frame_equal(batchCodeTable, batchCodeTableExpected, check_dtype = False)

    def createDataFrame(self, index, columns, data, dtypes = {}):
        return pd.DataFrame(index = index, columns = columns, data = data).astype(dtypes)


In [ ]:
from pandas.testing import assert_frame_equal

class DoseAnalysisTest(unittest.TestCase):

    def test_getDoseTable(self):
        # Given
        dataFrame = self.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
            data = [  [1,      0,          0,         'COVID19',  'MODERNA',  '016M20A', '2'],
                      [1,      0,          0,         'COVID19',  'MODERNA',  '030L20A', '1'],
                      [1,      1,          1,         'COVID19',  'MODERNA',  '030L20B', '1']],
            index = [
                "1048786",
                "1048786",
                "4711"],
                dtypes = {'VAX_DOSE_SERIES': "string"})
        dataFrame = DataFrameFilter().filterByCovid19(dataFrame)
        
        # When
        doseTable = DoseAnalysis.getDoseTable(dataFrame)

        # Then
        assert_frame_equal(
            doseTable,
            pd.DataFrame(
                data = {
                    'Total reports':              [2,                   1],
                    'Deaths':                     [2,                   1],
                    'Disabilities':               [1,                   0],
                    'Life Threatening Illnesses': [1,                   0],
                    'Severe reports (%)':         [(2 + 1 + 1)/2 * 100, (1 + 0 + 0)/1 * 100]
                },
                index = pd.Index(['1', '2'], dtype = "string", name = 'Dose')))
                
    def test_getDoseByMonthTable(self):
        # Given
        parseDate = lambda dateStr: pd.to_datetime(dateStr, format = "%m/%d/%Y")
        dataFrame = self.createDataFrame(
            columns = ['RECVDATE',             'DIED', 'L_THREAT', 'DISABLE', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
            data = [  [parseDate('01/01/2021'), 1,     0,          0,         'COVID19',  'MODERNA',  '016M20A', '2'],
                      [parseDate('01/01/2021'), 1,     0,          0,         'COVID19',  'MODERNA',  '030L20A', '1'],
                      [parseDate('01/01/2021'), 1,     1,          1,         'COVID19',  'MODERNA',  '030L20B', '1']],
            index = [
                "1048786",
                "1048786",
                "4711"],
            dtypes = {'VAX_DOSE_SERIES': "string"})
        dataFrame = DataFrameFilter().filterByCovid19(dataFrame)
        
        # When
        doseByMonthTable = DoseAnalysis.getDoseByMonthTable(dataFrame)

        # Then
        assert_frame_equal(
            doseByMonthTable,
            pd.DataFrame(
                data = {
                    'Total reports':              [2,                   1],
                    'Deaths':                     [2,                   1],
                    'Disabilities':               [1,                   0],
                    'Life Threatening Illnesses': [1,                   0],
                    'Severe reports (%)':         [(2 + 1 + 1)/2 * 100, (1 + 0 + 0)/1 * 100]
                },
                index = pd.MultiIndex.from_tuples(
                    [
                        (2021,  1, '1'),
                        (2021,  1, '2'),
                    ],
                names = ('Year', 'Month', 'Dose'))),
                check_index_type = False)

    def createDataFrame(self, index, columns, data, dtypes = {}):
        return pd.DataFrame(index = index, columns = columns, data = data).astype(dtypes)


In [ ]:
unittest.main(argv = [''], verbosity = 2, exit = False)

In [ ]:
def saveBatchCodeTable(manufacturer, excelFile):
    vaersDescrs = VaersDescrReader(dataDir = "VAERS").readAllVaersDescrs()
    dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(vaersDescrs)
    batchCodeTable = BatchCodeTableFactory.createBatchCodeTable(dataFrame, manufacturer = manufacturer, dose = '1')
    display(batchCodeTable)
    batchCodeTable.to_excel(excelFile)

### Moderna batch codes

In [ ]:
# https://www.howbadismybatch.com/moderna.html
saveBatchCodeTable("MODERNA", "results/moderna.xlsx")

### Pfizer batch codes

In [ ]:
# https://www.howbadismybatch.com/pfizer.html
saveBatchCodeTable("PFIZER\BIONTECH", "results/pfizer.xlsx")

### Janssen batch codes 

In [ ]:
# https://www.howbadismybatch.com/janssen.html
saveBatchCodeTable("JANSSEN", "results/janssen.xlsx")

### Short-list of 2000 batches having severe effects

In [ ]:
def saveSevereEffectsBatchCodeTable(excelFile):
    vaersDescrs = VaersDescrReader(dataDir = "VAERS").readAllVaersDescrs()
    dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(vaersDescrs)
    severeEffectsBatchCodeTable = BatchCodeTableFactory.createSevereEffectsBatchCodeTable(dataFrame, dose = '1')
    display(severeEffectsBatchCodeTable)
    severeEffectsBatchCodeTable.to_excel(excelFile)

In [ ]:
saveSevereEffectsBatchCodeTable('results/severeEffects.xlsx')

### Variation in Effect of First and Second Doses

In [ ]:
# https://www.howbadismybatch.com/firstsecond.html

def getDoseTable():
    vaersDescrs = VaersDescrReader(dataDir = "VAERS").readAllVaersDescrs()
    dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(vaersDescrs)
    dataFrame = DataFrameFilter().filterByCovid19(dataFrame)
    return DoseAnalysis.getDoseTable(dataFrame)

def getDoseByMonthTable():
    vaersDescrs = VaersDescrReader(dataDir = "VAERS").readAllVaersDescrs()
    dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(vaersDescrs)
    dataFrame = DataFrameFilter().filterByCovid19(dataFrame)
    return DoseAnalysis.getDoseByMonthTable(dataFrame)

In [ ]:
getDoseTable()

In [ ]:
doseByMonthTable = getDoseByMonthTable()
doseByMonthTable.to_excel('results/doseByMonthTable.xlsx')
doseByMonthTable